In [10]:
import os
from glob import glob
from PIL import Image
import numpy as np
from tqdm import tqdm

In [16]:
# PNG 파일이 들어있는 폴더 경로
input_folder = '/home/user/WindowsShare/05. Data/02. Training&Test/014. AI-Hub/콘크리트_콘크리트균열_원천_34'
output_folder = '/home/user/WindowsShare/05. Data/02. Training&Test/014. AI-Hub/콘크리트_콘크리트균열_원천_34/overlap/train'
alpha = 0.8

In [17]:
gt_list = glob(os.path.join(input_folder, "gtFine", "train", "*_gtFine_labelIds.png"))

# Define a dictionary for label colors
label_colors = {
    1: np.array([255, 0, 0]),   # Red for label 1
    2: np.array([0, 255, 0]),   # Green for label 2
    3: np.array([0, 255, 255]),
    4: np.array([255, 0, 255]),
}
    
for gt_path in tqdm(gt_list):

    # img 불러오기
    gt = Image.open(gt_path)
    gt_array = np.array(gt)

    # img 불러오기
    img_path = gt_path.replace('gtFine_labelIds.png', 'leftImg8bit.png')
    img_path = img_path.replace('gtFine', 'leftImg8bit')

    # Check if the img_path exists
    if os.path.exists(img_path):
        img = Image.open(img_path)
        seg_result = np.array(img)
        img_array = np.array(img)

        for label, color in label_colors.items():
            # Create mask for the current label
            mask_bool = gt_array == label

            # Blend the original image with the color based on the mask and alpha value
            seg_result[mask_bool] = seg_result[mask_bool] * (1 - alpha) + color * alpha

        # Convert the array back to an image
        blended_img = Image.fromarray(seg_result.astype('uint8'))

        # Save the blended image
        basename = os.path.basename(gt_path)
        result_path = os.path.join(output_folder, basename.replace('gtFine_labelIds', 'blended'))
        blended_img.save(result_path, "png")
    else:
        # Skip this file if img_path does not exist
        pass

100%|██████████| 249/249 [04:35<00:00,  1.11s/it]


In [9]:
img_list = glob(os.path.join(input_folder, "leftImg8bit", "train", "kaggle*"))
    
for img_path in tqdm(img_list):

    # img 불러오기
    img = Image.open(img_path)
    img_array = np.array(img)

    # gt 불러오기
    gt_path = img_path.replace('leftImg8bit.png', 'gtFine_labelIds.png')
    gt_path = gt_path.replace('leftImg8bit', 'gtFine')

    gt = Image.open(gt_path)
    gt_array = np.array(gt)

    # gt에서 라벨링된 index 구하기
    GA_index = np.where(gt_array==1)
    GA_x, GA_y = GA_index

    # 원본 이미지 픽셀값 불러오기
    px = img.load()

    # 라벨링된 부분 픽셀값 변경하기
    for i in range(len(GA_x)):
        px[int(GA_y[i]), int(GA_x[i])] = (255, 0, 0)

    # 변경된 이미지 저장하기
    basename = os.path.basename(gt_path)
    result_path = os.path.join(output_folder, basename)
    img.save(result_path, "png")

100%|██████████| 171/171 [07:57<00:00,  2.79s/it]


In [3]:
# Assuming input_folder, output_folder, and alpha are defined
img_list = glob(os.path.join(input_folder, "leftImg8bit", "val", "*leftImg8bit.png"))

# Define a dictionary for label colors
label_colors = {
    1: np.array([255, 0, 0]),   # Red for label 1
    2: np.array([0, 255, 0]),   # Green for label 2
    3: np.array([0, 255, 255]),
    4: np.array([255, 0, 255]),
    7: np.array([255, 255, 0]),
    8: np.array([0, 0, 255])
}

for img_path in tqdm(img_list):
    # Load image and convert to NumPy array
    img = Image.open(img_path)
    seg_result = np.array(img)

    # Load ground truth
    gt_path = img_path.replace('leftImg8bit.png', 'gtFine_labelIds.png')
    gt_path = gt_path.replace('leftImg8bit', 'gtFine')
    gt = Image.open(gt_path)
    gt_array = np.array(gt)

    for label, color in label_colors.items():
        # Create mask for the current label
        mask_bool = gt_array == label

        # Blend the original image with the color based on the mask and alpha value
        seg_result[mask_bool] = seg_result[mask_bool] * (1 - alpha) + color * alpha

    # Convert the array back to an image
    blended_img = Image.fromarray(seg_result.astype('uint8'))

    # Save the blended image
    basename = os.path.basename(gt_path)
    result_path = os.path.join(output_folder, basename)
    blended_img.save(result_path, "png")

100%|██████████| 98/98 [07:18<00:00,  4.47s/it]


In [4]:
# PNG 파일이 들어있는 폴더 경로
img_folder = '/home/user/WindowsShare/05. Data/00. Benchmarks/22. KhanhhaCrack/05. GAPS/leftImg8bit/val'
gt_folder = '/home/user/WEAK_DA/weak-da-seg/result/02. gaps_s-gaps_t-baseline-1031'  # GAPS384_test_0005_541_641_leftImg8bit_color.png
output_folder = '/home/user/WEAK_DA/weak-da-seg/result/gaps_gaps_overlap'

In [9]:
img_list = glob(os.path.join(img_folder, "*leftImg8bit.png"))

for img_path in tqdm(img_list):

    # img 불러오기
    img = Image.open(img_path)
    img_array = np.array(img)

    img_name = os.path.basename(img_path)
    gt_name = img_name.replace("_leftImg8bit.png", "_leftImg8bit_color.png")

    gt_path = os.path.join(gt_folder, gt_name)

    gt = Image.open(gt_path)
    gt_array = np.array(gt)

    # gt에서 라벨링된 index 구하기
    GA_index = np.where(gt_array==1)
    GA_x, GA_y = GA_index

    # 원본 이미지 픽셀값 불러오기
    px = img.load()

    # 라벨링된 부분 픽셀값 변경하기
    for i in range(len(GA_x)):
        px[int(GA_y[i]), int(GA_x[i])] = (255, 0, 0)

    # 변경된 이미지 저장하기
    basename = os.path.basename(gt_path)
    result_path = os.path.join(output_folder, basename)
    img.save(result_path, "png")

100%|██████████| 76/76 [00:04<00:00, 18.49it/s]
